The following cell sets the input values required to run the the optimisation model ``..\MINIZINC\hydrogen_plant.mzn``.

This input data are stored in ``..\MINIZINC\hydrogen_plant_data.dzn``. This input file preparation is carried out by ``make_dzn_file`` function that is in ``..\PYTHON\PACKAGE\optimisation.py``.

Then the minizinc optimisation is executed by ``optimise()`` function that is in ``..\PYTHON\PACKAGE\optimisation.py``. 

``projdirs`` stores the key paths for the entire package.

In [86]:
import pandas as pd
import numpy as np
import os
from projdirs import datadir #load the path that contains the data files 
from PACKAGE.optimisation import make_dzn_file, optimise


#Load the weather data file and extract wind speed and solar irradiance from it
DATA = pd.read_csv(datadir + r'TEST DATA\SAM_60min\086282TMY_60min_SAM.csv',skiprows=2)
solar = DATA['global horizontal irradiance'][0:1000].copy().tolist()
wind = DATA['wind speed'][0:1000].copy().tolist()

# create a dictionary that contains the inputs for optimisation.
#these inputs are used by make_dzn_file function to create an input text file called hydrogen_plant_data.dzn. 
simparams = dict(DT = 3600,#[s] time steps
                 ETA_EL = 0.80,       #efficiency of electrolyser
                 BAT_ETA_in = 0.95,   #charging efficiency of battery
                 BAT_ETA_out = 0.95,  #discharg efficiency of battery
                 C_PV = 0.9,          #[$/W] unit cost of PV
                 C_W = 1.2,           #[$/W] unit cost of W
                 C_EL = 0.5,          #[$/W] unit cost of electrolyser
                 C_HS = 16,           #[$/kgH] unit cost of hydrogen storage
                 C_BAT = 50/1000/3600,        #[$/W.s] unit cost of battery storage
                 CF = 0.99,           #capacity factor
                 S = solar,           #[W/m2] solar irradiance time series
                 W = wind,            #wind speed timeseries
                 L = [0.1 for i in range(len(solar))])       #[kgH/s] load profile timeseries

#run the optimisation function and get the results in a dictionary:
results = optimise(simparams)


print (
        'CAPEX = %0.2f [M$]' %( results['CAPEX'][0]/1e6 ),'\n'
        'pv_max = %0.2f [MW]' %( results['pv_max'][0]/1e6 ),'\n'  
        'w_max = %0.2f [MW]' %( results['w_max'][0]/1e6 ),'\n'    
        'el_max = %0.4f [MW]' %( results['el_max'][0]/1e6 ),'\n'  
        'hs_max = %0.2f [T of H2]' %( results['hs_max'][0]/1e3 ),'\n'  
        'bat_capa = %0.2f [MWh]' %( results['bat_capa'][0]/3.6e9  ),'\n'  
        'bat_pmax = %0.2f [MW]' %( results['bat_pmax'][0]/1e6 )
        )


#transfer the data from 'results' to a dataframe for plotting: 
data_plot = pd.DataFrame(dict(t= np.arange(0,len(simparams['S'])),
                              solar=simparams['S'],
                              wind=simparams['W'],
                              pv_out=results['pv_out']/1e6, #MW
                              w_out=results['w_out']/1e6, #MW
                              pc=results['pc']/1e6, #MW
                              he_out = results['he_out']*3.6, #T H2/hr
                              hr_out = results['hr_out']*3.6, #T H2/hr
                              sh = results['sh'][0:-1]/1e3, #T H2
                              R = results['r'][0:-1]/1e3, #T H2/hr 
                              L = results['L']*3.6, #T H2/hr
                              hs_out = (results['L']-results['hr_out'])*3.6, #T H2/hr
                              bat_e_stored = results['bat_e_stored'][0:-1]*3.6, #MWh
                              bat_pin = results['bat_pin']/1e6, #MW
                              bat_pout = results['bat_pout']/1e6 #MW
                             ))

#create a new column for datetime for plotting. This column is timestamped from the 
#beginning of 2022
data_plot['time'] =pd.to_datetime('2022-01-01') + pd.to_timedelta(data_plot.t, 'h')
data_plot.drop('t',axis=1,inplace=True)



CAPEX = 166.91 [M$] 
pv_max = 93.99 [MW] 
w_max = 53.04 [MW] 
el_max = 28.4980 [MW] 
hs_max = 9.74 [T of H2] 
bat_capa = 84.82 [MWh] 
bat_pmax = 66.71 [MW]


### plot hydrogen flow

In [87]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure
# fig = go.Figure()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.sh),
              name='Stored H',yaxis="y1"),
                )
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.R),
              name='R',yaxis="y1")
                )
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.he_out),
              line_shape='hv',name='he_out',yaxis="y2")
                )
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.hs_out),
              line_shape='hv',name='hs_out',yaxis="y2")
                )
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.hr_out),
              line_shape='hv',name='hr_out',yaxis="y2")
                )
# fig.update_layout(
#     yaxis1=dict(
#         range=[0, max(data_plot.sh)]
#     ),
#     yaxis2=dict(
#         range=[0, 0.4]
#     )   
# )
fig.update_yaxes(dict(title = 'T of H2'), secondary_y=False)
fig.update_yaxes(dict(title = 'T of H2/hr'), secondary_y=True)
fig.show()

### plot power flow

In [88]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# Create figure
# fig = go.Figure()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.pv_out),
              line_shape='hv', name='pv_out'),
                )
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.w_out),
              line_shape='hv', name='w_out'),
                )
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.pc),
              line_shape='hv', name='pc'),
                )
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.bat_pin),
              line_shape='hv', name='Bat. Pin'),
                )
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.bat_pout),
              line_shape='hv', name='Bat. Pout'),
                )
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.bat_e_stored),
              line_shape='hv', name='battery energy', yaxis="y2"),
                )
fig.update_yaxes(dict(title = 'MW'), secondary_y=False)
fig.update_yaxes(dict(title = 'MWh'), secondary_y=True)
fig.show()

In [76]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.solar),
              line_shape='hv', name='Solar Irrad. [W/m<sup>2</sup>]', yaxis="y1"),
                )
fig.add_trace(
    go.Scatter(x=list(data_plot.time), y=list(data_plot.wind),
              line_shape='hv', name='Wind Speed [m/s]', yaxis="y2"),
                )
fig.update_yaxes(dict(title = 'W/m<sup>2</sup>'), secondary_y=False)
fig.update_yaxes(dict(title = 'm/s'), secondary_y=True)
fig.show()